In [17]:
from tabulate import tabulate

In [15]:
# Constants
ENERGY_CONSUMPTION = "Energy Consumption (exc. PUE)"
ENERGY_CONSUMPTION_PUE = "Energy Consumption (inc. PUE)"
MEMORY_CONSUMPTION = "Memory Energy Consumption (exc. PUE)"
MEMORY_CONSUMPTION_PUE = "Memory Energy Consumption (inc. PUE)"
CARBON_EMISSIONS = "Operational Carbon Emissions"
EMBODIED_CARBON_EMISSIONS = "Embodied Carbon Emissions"
TOTAL_CARBON_EMISSIONS = "Total Carbon Emissions"
RES_MEM_ENERGY_CONSUMPTION = "Reserved Memory Energy Consumption"
RES_MEM_CARBON_EMISSIONS = "Reserved Memory Carbon Emissions"

In [16]:
# Parse Key Details from Summary Files
def get_data(lines):
    data = {}

    for line in lines:
        parts = line.strip().split(':')
        parts[0] = parts[0][2:].strip()
        data[parts[0]] = parts[1].replace('kWh', '').replace('gCO2e', '').strip()

    return data


def get_bonus_data(lines):
    data = {}

    for line in lines:
        if ':' in line: 
            parts = line.strip().split(':')

            if parts[0] == 'Task Runtime':
                continue

            data[parts[0]] = parts[1].replace('kWh', '').replace('gCO2e', '').strip()
        else:
            if '|' in line:
                cpu_mem_split = line.split('|')
                cpu = cpu_mem_split[0].strip()
                mem = cpu_mem_split[1].strip()

    data['cpu'] = cpu.replace('% CPU [', '').replace('%]', '')
    data['mem'] = mem.replace('% Memory [', '').replace('%]', '')

    return data


def parse_summary(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    info = get_data(lines[2:5])
    data = get_data(lines[7:14])
    bonus = get_bonus_data(lines[14:])

    return (info, data, bonus)


def get_average(first, second, third):
    data = {}

    for key in first.keys():
        data[key] = (float(first[key]) + float(second[key]) + float(third[key])) / 3

    return data


def print_summary(data):
    print(f"{ENERGY_CONSUMPTION}: {data[ENERGY_CONSUMPTION]:.2f}kWh")
    print(f"{ENERGY_CONSUMPTION_PUE}: {data[ENERGY_CONSUMPTION_PUE]:.2f}kWh")
    print(f"{MEMORY_CONSUMPTION}: {data[MEMORY_CONSUMPTION]:.2f}kWh")
    print(f"{MEMORY_CONSUMPTION_PUE}: {data[MEMORY_CONSUMPTION_PUE]:.2f}kWh")
    print(f"{CARBON_EMISSIONS}: {data[CARBON_EMISSIONS]:.2f}gCO2e")
    print(f"{EMBODIED_CARBON_EMISSIONS}: {data[EMBODIED_CARBON_EMISSIONS]:.2f}gCO2e")
    print(f"{TOTAL_CARBON_EMISSIONS}: {data[TOTAL_CARBON_EMISSIONS]:.2f}gCO2e")

    return {
        ENERGY_CONSUMPTION: data[ENERGY_CONSUMPTION],
        MEMORY_CONSUMPTION: data[MEMORY_CONSUMPTION],
        CARBON_EMISSIONS: data[CARBON_EMISSIONS],
        EMBODIED_CARBON_EMISSIONS: data[EMBODIED_CARBON_EMISSIONS],
        TOTAL_CARBON_EMISSIONS: data[TOTAL_CARBON_EMISSIONS]
    }


def print_bonus(data):
    print(f"{RES_MEM_ENERGY_CONSUMPTION}: {data[RES_MEM_ENERGY_CONSUMPTION]:.2f}kWh")
    print(f"{REST_MEM_CARBON_EMISSIONS}: {data[REST_MEM_CARBON_EMISSIONS]:.2f}gCO2e")
    print(f"CPU [{data['cpu']:.2f}% | {data['mem']:.2f}%] Memory")

    return {
        RES_MEM_ENERGY_CONSUMPTION: data[RES_MEM_ENERGY_CONSUMPTION],
        REST_MEM_CARBON_EMISSIONS: data[REST_MEM_CARBON_EMISSIONS]
    }


def print_info(data_1, data_2, data_3):
    matches = data_1 == data_2 == data_3

    if not matches:
        print("[ERROR] Workflow Data does not match ...")
        return

    print(f"INFO: CPU Energy Consumption [{data_1['power model selected']}] | PUE [{data_1['power-usage-effectiveness']}] | Memory Energy Consumption [{data_1['memory-power-draw']} W/GB]") 


def report_summary_for(filename_1, filename_2, filename_3, bonus=False):
    (info_1, data_1, bonus_1) = parse_summary(filename_1)
    (info_2, data_2, bonus_2) = parse_summary(filename_2)
    (info_3, data_3, bonus_3) = parse_summary(filename_3)
    avg_data = get_average(data_1, data_2, data_3)

    if bonus:
        avg_bonus = get_average(bonus_1, bonus_2, bonus_3)
        print_info(info_1, info_2, info_3)
        summary = print_summary(avg_data)
        bonus = print_bonus(avg_bonus)
        summary.update(bonus)
    else:
        summary = print_summary(avg_data)

    return summary


In [5]:
# Trace File Paths List
workflows = ['chipseq', 'mag', 'montage', 'nanoseq', 'rangeland', 'rnaseq', 'sarek']
memory = [128, 256, 128, 32, 256, 128, 128]
no_nodes = [8, 8, 8, 1, 8, 8, 8]
resource_settings = ['clusterb_default_minmax', 'hu26_default_minmax', 'clusterb_default_minmax', 'server_default_minmax', 'hu26_default_minmax', 'clusterb_default_minmax', 'clusterb_default_minmax']
cis = ['de-15112023-08122023', 'de-09052024-01082024', 'de-01012024-31012024', 'gb-31082024-30092024', 'de-09052024-01082024', 'de-15112023-08122023', 'de-15112023-08122023']
workflow_baseline_output = '../data/results/baseline/out/'

In [12]:
# Display Data for Each Average Workflow
avg_workflow_stats = {}
for workflow, ci, setting, node_memory, nodes in zip(workflows, cis, resource_settings, memory, no_nodes):
    print(f'\n{workflow}:')
    print(f'INFO: Node Memory [{node_memory} GB] | No. Nodes [{nodes}]')
    summary_files = [f'{workflow_baseline_output}{workflow}-{iteration}-{ci}-{setting}-summary.txt' for iteration in range(1, 4)]
    avg_workflow_stats[workflow] = report_summary_for(summary_files[0], summary_files[1], summary_files[2], bonus=True)


chipseq:
INFO: Node Memory [128 GB] | No. Nodes [8]
INFO: CPU Energy Consumption [clusterb_default_minmax] | PUE [1.0] | Memory Energy Consumption [0.392 W/GB]
Energy Consumption (exc. PUE): 22.68kWh
Energy Consumption (inc. PUE): 22.68kWh
Memory Energy Consumption (exc. PUE): 0.52kWh
Memory Energy Consumption (inc. PUE): 0.52kWh
Operational Carbon Emissions: 12795.43gCO2e
Embodied Carbon Emissions: 95.05gCO2e
Total Carbon Emissions: 12890.48gCO2e
Reserved Memory Energy Consumption: 1.49kWh
Reserved Memory Carbon Emissions: 817.55gCO2e
CPU [91.88% | 8.12%] Memory

mag:
INFO: Node Memory [256 GB] | No. Nodes [8]
INFO: CPU Energy Consumption [hu26_default_minmax] | PUE [1.0] | Memory Energy Consumption [0.392 W/GB]
Energy Consumption (exc. PUE): 30.38kWh
Energy Consumption (inc. PUE): 30.38kWh
Memory Energy Consumption (exc. PUE): 2.59kWh
Memory Energy Consumption (inc. PUE): 2.59kWh
Operational Carbon Emissions: 6649.07gCO2e
Embodied Carbon Emissions: 170.55gCO2e
Total Carbon Emissions

In [13]:
# Display Data for Each Average Workflow
marg_workflow_stats = {}
for workflow, ci, setting, node_memory, nodes in zip(workflows, cis, resource_settings, memory, no_nodes):
    print(f'\n{workflow}:')
    print(f'INFO: Node Memory [{node_memory} GB] | No. Nodes [{nodes}]')
    summary_files = [f'{workflow_baseline_output}{workflow}-{iteration}-{ci}-marg-{setting}-summary.txt' for iteration in range(1, 4)]
    marg_workflow_stats[workflow] = report_summary_for(summary_files[0], summary_files[1], summary_files[2], bonus=True)


chipseq:
INFO: Node Memory [128 GB] | No. Nodes [8]
INFO: CPU Energy Consumption [clusterb_default_minmax] | PUE [1.0] | Memory Energy Consumption [0.392 W/GB]
Energy Consumption (exc. PUE): 23.76kWh
Energy Consumption (inc. PUE): 23.76kWh
Memory Energy Consumption (exc. PUE): 0.55kWh
Memory Energy Consumption (inc. PUE): 0.55kWh
Operational Carbon Emissions: 18823.88gCO2e
Embodied Carbon Emissions: 99.79gCO2e
Total Carbon Emissions: 18923.67gCO2e
Reserved Memory Energy Consumption: 1.49kWh
Reserved Memory Carbon Emissions: 1148.48gCO2e
CPU [92.11% | 7.89%] Memory

mag:
INFO: Node Memory [256 GB] | No. Nodes [8]
INFO: CPU Energy Consumption [hu26_default_minmax] | PUE [1.0] | Memory Energy Consumption [0.392 W/GB]
Energy Consumption (exc. PUE): 30.39kWh
Energy Consumption (inc. PUE): 30.39kWh
Memory Energy Consumption (exc. PUE): 2.59kWh
Memory Energy Consumption (inc. PUE): 2.59kWh
Operational Carbon Emissions: 23261.04gCO2e
Embodied Carbon Emissions: 170.64gCO2e
Total Carbon Emissio

In [19]:
avg_data_rows = []
data_head = ['workflow', 'cpu_energy', 'mem_energy', 'cpu_mem_ems', 'node_mem_energy', 'node_mem_ems', 'emb_ems', 'total_ems']
for workflow in workflows:
    wf_data = avg_workflow_stats[workflow] 
    total_ems = wf_data[CARBON_EMISSIONS] + wf_data[RES_MEM_CARBON_EMISSIONS] + wf_data[EMBODIED_CARBON_EMISSIONS]
    row = [workflow, wf_data[ENERGY_CONSUMPTION], wf_data[MEMORY_CONSUMPTION], wf_data[CARBON_EMISSIONS], wf_data[RES_MEM_ENERGY_CONSUMPTION], wf_data[RES_MEM_CARBON_EMISSIONS], wf_data[EMBODIED_CARBON_EMISSIONS], total_ems]
    avg_data_rows.append(row)

print('avg emissions')
print(tabulate(avg_data_rows, data_head, tablefmt='orgtbl'))

avg emissions
| workflow   |   cpu_energy |   mem_energy |   cpu_mem_ems |   node_mem_energy |   node_mem_ems |   emb_ems |   total_ems |
|------------+--------------+--------------+---------------+-------------------+----------------+-----------+-------------|
| chipseq    |    22.6803   |    0.519725  |     12795.4   |         1.48566   |      817.547   |  95.0484  |  13708      |
| mag        |    30.3773   |    2.58726   |      6649.07  |        10.1414    |     2049.77    | 170.548   |   8869.39   |
| montage    |     0.930141 |    0.0155522 |       510.29  |         0.211891  |      114.311   |   4.33696 |    628.937  |
| nanoseq    |     0.307432 |    0.046262  |        30.288 |         0.0659505 |        5.75203 |   2.25864 |     38.2987 |
| rangeland  |     9.71625  |    1.46464   |      2724.57  |         5.53036   |     1379.3     |  56.1914  |   4160.06   |
| rnaseq     |    14.1179   |    0.585663  |      3832.38  |         1.22771   |      319.138   |  60.8424  |   4212.3

In [20]:
marg_data_rows = []
data_head = ['workflow', 'cpu_energy', 'mem_energy', 'cpu_mem_ems', 'node_mem_energy', 'node_mem_ems', 'emb_ems', 'total_ems']

for workflow in workflows:
    wf_data = marg_workflow_stats[workflow] 
    total_ems = wf_data[CARBON_EMISSIONS] + wf_data[RES_MEM_CARBON_EMISSIONS] + wf_data[EMBODIED_CARBON_EMISSIONS]
    row = [workflow, wf_data[ENERGY_CONSUMPTION], wf_data[MEMORY_CONSUMPTION], wf_data[CARBON_EMISSIONS], wf_data[RES_MEM_ENERGY_CONSUMPTION], wf_data[RES_MEM_CARBON_EMISSIONS], wf_data[EMBODIED_CARBON_EMISSIONS], total_ems]
    marg_data_rows.append(row)

print('marg emissions')
print(tabulate(marg_data_rows, data_head, tablefmt='orgtbl'))

marg emissions
| workflow   |   cpu_energy |   mem_energy |   cpu_mem_ems |   node_mem_energy |   node_mem_ems |   emb_ems |   total_ems |
|------------+--------------+--------------+---------------+-------------------+----------------+-----------+-------------|
| chipseq    |    23.7603   |    0.548735  |     18823.9   |         1.48566   |      1148.48   |  99.7885  |   20072.1   |
| mag        |    30.3929   |    2.5862    |     23261     |        10.1414    |      7177.31   | 170.637   |   30609     |
| montage    |     1.11303  |    0.0211917 |       860.26  |         0.211891  |       160.415  |   5.18624 |    1025.86  |
| nanoseq    |     0.307765 |    0.0463203 |       142.051 |         0.0626244 |        25.1344 |   2.26163 |     169.447 |
| rangeland  |     9.72082  |    1.4651    |      7901.27  |         5.52984   |      3988.51   |  56.2176  |   11946     |
| rnaseq     |    14.5713   |    0.600466  |     11698.4   |         1.22771   |       945.456  |  62.8145  |   12706